In [ ]:
!pip install LanusStats

In [ ]:
import LanusStats as ls
import pandas as pd
import re
import glob
from bs4 import BeautifulSoup

# Scrapping Estadísticas de partidos

In [ ]:
fbref = ls.Fbref()

In [ ]:

base_path = "https://fbref.com/"

def extract_match_links(file_path):
    # Open and read the HTML file
    with open(file_path, "r", encoding="utf-8") as file:
        soup = BeautifulSoup(file, "html.parser")

    # Find all <td> elements with class "center" and data-stat="score"
    td_tags = soup.find_all("td", class_="left", attrs={"data-stat": "match_report"})

    links = []
    for td in td_tags:
        a_tag = td.find("a")
        if a_tag and a_tag.has_attr("href"):
            links.append(base_path + a_tag["href"])

    return links

links = extract_match_links("partidos_18_19.html")

In [ ]:
print(links)

['https://fbref.com//en/matches/82b4ca95/Girona-Valladolid-August-17-2018-La-Liga', 'https://fbref.com//en/matches/9fe3febe/Real-Betis-Levante-August-17-2018-La-Liga', 'https://fbref.com//en/matches/a8d60ff6/Celta-Vigo-Espanyol-August-18-2018-La-Liga', 'https://fbref.com//en/matches/aeeb77fc/Villarreal-Real-Sociedad-August-18-2018-La-Liga', 'https://fbref.com//en/matches/e95b2588/Barcelona-Alaves-August-18-2018-La-Liga', 'https://fbref.com//en/matches/446f0d99/Eibar-Huesca-August-19-2018-La-Liga', 'https://fbref.com//en/matches/d1aa3271/Rayo-Vallecano-Sevilla-August-19-2018-La-Liga', 'https://fbref.com//en/matches/25f5e47e/Real-Madrid-Getafe-August-19-2018-La-Liga', 'https://fbref.com//en/matches/1202acb1/Valencia-Atletico-Madrid-August-20-2018-La-Liga', 'https://fbref.com//en/matches/95ac7d44/Athletic-Club-Leganes-August-20-2018-La-Liga', 'https://fbref.com//en/matches/aed2b90b/Getafe-Eibar-August-24-2018-La-Liga', 'https://fbref.com//en/matches/75220ead/Leganes-Real-Sociedad-August-2

In [ ]:
data = fbref.get_all_dfs("https://fbref.com//en/matches/056f7848/Eibar-Celta-Vigo-September-12-2020-La-Liga")

In [ ]:
season_data = []
for idx, match_link in enumerate(links):
  print(f"Procesando partido {idx}")
  data = fbref.get_all_dfs(match_link)

  datos_partido = data[2]
  df = pd.DataFrame(datos_partido)

  home_team = df.columns[0][0]
  away_team = df.columns[1][0]

  home_data = df.iloc[0::2, 0][:4]
  home_data = [home_team, home_data[0], home_data[2], home_data[4], home_data[6]]
  away_data = df.iloc[0::2, 1][:4]
  away_data = [away_team, away_data[0], away_data[2], away_data[4], away_data[6]]

  season_data.append([home_data, away_data])

Procesando partido 0
Procesando partido 1
Procesando partido 2


HTTPError: HTTP Error 404: Not Found

In [ ]:
structured_data = []
for match in season_data:
    home_team, home_possession, home_passes, home_shots, home_saves = match[0]
    away_team, away_possession, away_passes, away_shots, away_saves = match[1]

    structured_data.append({
        "home": home_team,
        "away": away_team,
        "posesión_home": home_possession,
        "posesión_away": away_possession,
        "pases_home": home_passes,
        "pases_away": away_passes,
        "disparos_home": home_shots,
        "disparos_away": away_shots,
    })

# Creating DataFrame
df = pd.DataFrame(structured_data)
df.to_csv("estadisticas_23_24.csv")

# Agregado de datos

In [ ]:
def split_shots_column(csv_file, team, exp):
    df = csv_file

    # Function to extract shots, shots_on_target, and pct_shots_on_target
    def extract_shots_data(value):
        match = re.match(exp, str(value))
        if match:
          if team == "home":
            shots_on_target, shots, pct_shots_on_target = match.groups()
            return int(shots), int(shots_on_target), (int(pct_shots_on_target) / 100)
          else:
            pct_shots_on_target, shots_on_target, shots  = match.groups()
            return int(shots), int(shots_on_target), (int(pct_shots_on_target) / 100)
        return None, None, None  # Handle missing or malformed data

    df[[f"{team}_shots", f"{team}_shots_on_target", f"{team}_shot_accuracy"]] = df[f"disparos_{team}"].apply(lambda x: pd.Series(extract_shots_data(x)))

    df.drop(columns=[f"disparos_{team}"], inplace=True)

    return df

def split_passes_column(csv_file, team, exp):
    df = csv_file

    def extract_passes_data(value):
        match = re.match(exp, str(value))
        if match:
          if team == "home":
            completed_passes, passes, passing_accuracy = match.groups()
            return int(passes), int(completed_passes), (int(passing_accuracy) / 100)
          else:
            passing_accuracy, completed_passes, passes  = match.groups()
            return int(passes), int(completed_passes), (int(passing_accuracy) / 100)
        return None, None, None  # Handle missing or malformed data

    # Apply extraction function to "disparos_home" column
    df[[f"{team}_passes", f"{team}_completed_passes", f"{team}_passing_accuracy"]] = df[f"pases_{team}"].apply(lambda x: pd.Series(extract_passes_data(x)))

    df.drop(columns=[f"pases_{team}"], inplace=True)

    return df

In [ ]:
def merge_csv_columns(partidos_csv, estadisticas_csv, columns_to_add):
    # Load the CSV files into DataFrames
    source_df = pd.read_csv(estadisticas_csv, usecols=columns_to_add)
    target_df = pd.read_csv(partidos_csv)

    # Ensure both DataFrames have the same number of rows
    if len(source_df) != len(target_df):
        raise ValueError("The two CSV files must have the same number of rows.")

    # Concatenate columns from source_df to target_df
    merged_df = pd.concat([target_df, source_df], axis=1)

    # drop unused columns
    # merged_df.drop(columns=["salvadas_home", "salvadas_away"], inplace=True)

    # cast percentage to float
    merged_df["posesión_home"] = merged_df["posesión_home"].str.rstrip("%").astype(float) / 100
    merged_df["posesión_away"] = merged_df["posesión_away"].str.rstrip("%").astype(float) / 100

    # Save the result to a new CSV file
    return merged_df

In [ ]:
data = merge_csv_columns("la_liga_temp_18_19.csv", "estadisticas_18_19.csv" , ["posesión_home", "posesión_away", "pases_home", "pases_away", "disparos_home", "disparos_away"])
data = split_shots_column(data, "home", r"(\d+) of (\d+)\s+—\s+(\d+)%")
data = split_shots_column(data, "away", r"(\d+)%\s+—\s+(\d+)\s+of\s+(\d+)")
data = split_passes_column(data, "home", r"(\d+)\s+of\s+(\d+)\s+—\s+(\d+)%")
data = split_passes_column(data, "away", r"(\d+)%\s+—\s+(\d+)\s+of\s+(\d+)")
data.to_csv("la_liga_temp_18_19_ext.csv")

In [ ]:
# Combinar los resultados
def merge_csv_files(output_csv):
    # Get all CSV files in the folder
    csv_files = glob.glob("datasets/*.csv")

    if not csv_files:
        print("No CSV files found in the folder.")
        return

    df = pd.read_csv(csv_files[0])

    # Read the remaining CSV files without headers (use header=None and set column names)
    df_list = [df]  # Store the first DataFrame

    for file in csv_files[1:]:
        temp_df = pd.read_csv(file, header=None)  # Read without headers
        df_list.append(temp_df)  # Append to list

    # Concatenate all DataFrames
    merged_df = pd.concat(df_list, ignore_index=True)

    # Save the merged DataFrame to a new CSV file
    merged_df.to_csv(output_csv, index=False)

    print(f"Successfully merged {len(csv_files)} CSV files into {output_csv}")

In [ ]:
merge_csv_files("la_liga_temp_19_24_ext.csv")

Successfully merged 5 CSV files into la_liga_temp_19_24_ext.csv
